In [1]:
import torch
import transformers
from datasets import load_from_disk

/usr/local/lib/crp-custom-env/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds_20_obf = load_from_disk("./data/openai_humaneval/humaneval_test_20_with_obf")
df = ds_20_obf.to_pandas()
df.head()

,task_id,prompt,canonical_solution,test,entry_point,prompt_original,prompt_low,prompt_high
0,HumanEval/0,from typing import List\n\n\ndef has_close_ele...,"for idx, elem in enumerate(numbers):\n ...","\n\nMETADATA = {\n 'author': 'jt',\n 'da...",has_close_elements,from typing import List\n\n\ndef has_close_ele...,from typing import List\n\ndef has_close_eleme...,from typing import List\n\ndef has_close_eleme...
1,HumanEval/1,from typing import List\n\n\ndef separate_pare...,result = []\n current_string = []\n ...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",separate_paren_groups,from typing import List\n\n\ndef separate_pare...,from typing import List\n\ndef separate_paren_...,from typing import List\n\ndef separate_paren_...
2,HumanEval/2,\n\ndef truncate_number(number: float) -> floa...,return number % 1.0\n,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",truncate_number,\n\ndef truncate_number(number: float) -> floa...,def truncate_number(arg0: float) -> float:\n ...,def truncate_number(arg0: float) -> float:\n ...
3,HumanEval/3,from typing import List\n\n\ndef below_zero(op...,balance = 0\n\n for op in operations:\n...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",below_zero,from typing import List\n\n\ndef below_zero(op...,from typing import List\n\ndef below_zero(arg0...,from typing import List\n\ndef below_zero(arg0...
4,HumanEval/4,from typing import List\n\n\ndef mean_absolute...,mean = sum(numbers) / len(numbers)\n re...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",mean_absolute_deviation,from typing import List\n\n\ndef mean_absolute...,from typing import List\n\ndef mean_absolute_d...,from typing import List\n\ndef mean_absolute_d...


In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

checkpoint = "Salesforce/codet5p-2b"
device = "cuda" if torch.cuda.is_available() else "cpu" # for GPU usage or "cpu" for CPU usage

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint,
                                              torch_dtype=torch.float16,
                                              trust_remote_code=True).to(device)

/home/coder/.conda/envs/ml/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
A new version of the following files was downloaded from https://huggingface.co/Salesforce/codet5p-2b:
- modeling_codet5p.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [5]:
variant = "prompt_original"
prompt = df.loc[0, variant]

print(prompt)

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """



In [15]:
python_code = "def fit_linear_regression(X, y):"

In [26]:
variant_2 = "prompt_low"
prompt = df.loc[0, variant_2]

In [ ]:
variant_3 = "prompt_high"
prompt = df.loc[0, variant_3]

In [30]:
encoding = tokenizer(prompt, return_tensors="pt").to(device)
encoding['decoder_input_ids'] = encoding['input_ids'].clone()
outputs = model.generate(**encoding, max_length=256)
answear = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(answear)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


from typing import List

def has_close_elements(arg0: List[float], arg1: float) -> bool:
        """
    Check if in given list of arg0, are any two arg0 closer to each other than given arg1.
    """
        for i in range(len(arg0)):
            for j in range(i+1, len(arg0)):
                if abs(arg0[i] - arg0[j]) < arg1:
                    return True
        return False

def has_close_elements_2(arg0: List[float], arg1: float) -> bool:
    """
    Check if in given list of arg0, are any two arg0 closer to each other than given arg1.
    """
        for i in range(len(arg0)):
            for j in range(i+1, len(arg0)):
                if abs(arg0[i] - arg0[j]) < arg1:
                    return True
        return False

def has_close_elements_3(arg0: List[float], arg1


In [21]:
import re

def extract_clean_function_body(generated: str) -> str:
    """
    Extract executable function body only:
    - removes repeated def
    - removes docstrings
    - stops at indentation break
    """

    # Remove regenerated def if present
    def_match = re.search(r"def\s+\w+\s*\(.*?\):", generated)
    if def_match:
        generated = generated[def_match.end():]

    lines = generated.splitlines()
    body_lines = []
    started = False
    in_docstring = False

    for line in lines:
        if not started:
            if line.strip() == "":
                continue
            if line.startswith("    ") or line.startswith("\t"):
                started = True
            else:
                continue

        stripped = line.lstrip()

        # Docstring toggle
        if stripped.startswith('"""') or stripped.startswith("'''"):
            in_docstring = not in_docstring
            continue

        if in_docstring:
            continue

        # Stop if indentation breaks
        if not (line.startswith("    ") or line.startswith("\t") or line.strip() == ""):
            break

        body_lines.append(line)

    body = "\n".join(body_lines).rstrip()
    return body + "\n" if body else ""

In [37]:
def codet5_generate_raw(prompt: str, model, tokenizer, device, max_length: int = 256) -> str:
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    encoding["decoder_input_ids"] = encoding["input_ids"].clone()
    outputs = model.generate(**encoding, max_length=max_length)
    answer = tokenizer.decode(outputs[0],skip_special_tokens=True)

    return answer

In [42]:
row_idx = 0
prompt = df.loc[row_idx, "prompt_original"]

raw = codet5_generate_raw(prompt, model, tokenizer, device)
body = extract_clean_function_body(raw)

print("===== RAW GENERATION =====")
print(raw)

print("\n===== CLEAN BODY =====")
print(body)

print("\n===== FULL FUNCTION =====")
print(prompt + body)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


===== RAW GENERATION =====
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """
    for i in range(len(numbers) - 1):
        if numbers[i + 1] - numbers[i] < threshold:
            return True
    return False
    

if __name__ == "__main__":
    import doctest
    doctest.testmod()
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    


===== CLEAN BODY =====
    for i in range(len(numbers) - 1):
        if numbers[i + 1] - numbers[i] < threshold:
            return True
    return False


===== FULL FUNCTION =====
from typing import List


def has_close_elements(numbers: List[float], threshold: floa

In [43]:
df["codet5_solution_original"] = ""

for i in range(len(df)):
    prompt = df.loc[i, "prompt_original"]
    raw = codet5_generate_raw(prompt, model, tokenizer, device)
    body = extract_clean_function_body(raw)
    df.loc[i, "codet5_solution_original"] = body

print("Generation complete.")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Generation complete.


In [47]:
df["codet5_solution_low"] = ""

for i in range(len(df)):
    prompt = df.loc[i, "prompt_low"]
    raw = codet5_generate_raw(prompt, model, tokenizer, device)
    body = extract_clean_function_body(raw)
    df.loc[i, "codet5_solution_low"] = body

print("Generation complete.")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Generation complete.


In [50]:
df["codet5_solution_high"] = ""

for i in range(len(df)):
    prompt = df.loc[i, "prompt_high"]
    raw = codet5_generate_raw(prompt, model, tokenizer, device)
    body = extract_clean_function_body(raw)
    df.loc[i, "codet5_solution_high"] = body

print("Generation complete.")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Generation complete.


In [51]:
df.head()

,task_id,prompt,canonical_solution,test,entry_point,prompt_original,prompt_low,prompt_high,codet5_solution_original,codet5_solution_low,codet5_solution_high
0,HumanEval/0,from typing import List\n\n\ndef has_close_ele...,"for idx, elem in enumerate(numbers):\n ...","\n\nMETADATA = {\n 'author': 'jt',\n 'da...",has_close_elements,from typing import List\n\n\ndef has_close_ele...,from typing import List\n\ndef has_close_eleme...,from typing import List\n\ndef has_close_eleme...,for i in range(len(numbers) - 1):\n ...,for i in range(len(arg0)):\n for j ...,for i in range(len(arg0)):\n ...
1,HumanEval/1,from typing import List\n\n\ndef separate_pare...,result = []\n current_string = []\n ...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",separate_paren_groups,from typing import List\n\n\ndef separate_pare...,from typing import List\n\ndef separate_paren_...,from typing import List\n\ndef separate_paren_...,"paren_string = paren_string.replace(' ', '...",paren_groups = []\n paren_stack = []\n ...,paren_stack = []\n paren_list =...
2,HumanEval/2,\n\ndef truncate_number(number: float) -> floa...,return number % 1.0\n,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",truncate_number,\n\ndef truncate_number(number: float) -> floa...,def truncate_number(arg0: float) -> float:\n ...,def truncate_number(arg0: float) -> float:\n ...,if number == 0:\n return 0\n if ...,return arg0 - int(arg0)\n,return arg0\n
3,HumanEval/3,from typing import List\n\n\ndef below_zero(op...,balance = 0\n\n for op in operations:\n...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",below_zero,from typing import List\n\n\ndef below_zero(op...,from typing import List\n\ndef below_zero(arg0...,from typing import List\n\ndef below_zero(arg0...,for operation in operations:\n if o...,if arg0[0] < 0:\n return True\n ...,if arg0[0] < 0:\n return Tr...
4,HumanEval/4,from typing import List\n\n\ndef mean_absolute...,mean = sum(numbers) / len(numbers)\n re...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",mean_absolute_deviation,from typing import List\n\n\ndef mean_absolute...,from typing import List\n\ndef mean_absolute_d...,from typing import List\n\ndef mean_absolute_d...,mean = sum(numbers) / len(numbers)\n re...,arg0_mean = sum(arg0) / len(arg0)\n ret...,return sum(abs(x - arg0[0]) for x in a...


In [58]:
print(df.iloc[4]["canonical_solution"])

    mean = sum(numbers) / len(numbers)
    return sum(abs(x - mean) for x in numbers) / len(numbers)



In [59]:
print(df.iloc[4]["codet5_solution_original"])

    mean = sum(numbers) / len(numbers)
    return sum(abs(x - mean) for x in numbers) / len(numbers)



In [60]:
print(df.iloc[4]["codet5_solution_low"])

    arg0_mean = sum(arg0) / len(arg0)
    return sum([abs(x - arg0_mean) for x in arg0]) / len(arg0)



In [57]:
print(df.iloc[4]["codet5_solution_high"])

        return sum(abs(x - arg0[0]) for x in arg0) / len(arg0)



In [61]:
out_path = "./data/openai_humaneval/humaneval_test_20_with_codet5_original.jsonl"

df.to_json(out_path, orient="records", lines=True)
print("Saved to:", out_path)

Saved to: ./data/openai_humaneval/humaneval_test_20_with_codet5_original.jsonl


In [62]:
print(model.device)

cuda:0


Let us try couple of more coder models for further ananlysis in this block to see how different trained models will behave on the codes in general

In [14]:
import torch
import re
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd

In [15]:
import pandas as pd

In [16]:
df = pd.read_json("./data/openai_humaneval/humaneval_test_20_with_codet5_original.jsonl", lines=True)

In [17]:
variant = "prompt_original"
prompt = df.loc[0, variant]

In [18]:
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-1.3b-base", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-1.3b-base", trust_remote_code=True).cuda()

Unrecognized keys in `rope_scaling` for 'rope_type'='linear': {'type'}


In [23]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_length=256)
answear = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(answear)

Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.


from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """
    for i in range(len(numbers)):
        for j in range(i + 1, len(numbers)):
            if abs(numbers[i] - numbers[j]) < threshold:
                return True
    return False


if __name__ == "__main__":
    import doctest
    doctest.testmod()


In [25]:
body = extract_clean_function_body(answear)
print(body)

    for i in range(len(numbers)):
        for j in range(i + 1, len(numbers)):
            if abs(numbers[i] - numbers[j]) < threshold:
                return True
    return False



In [26]:
def deepseek_generate_raw(prompt: str, model, tokenizer, device, max_length: int = 256) -> str:
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**encoding, max_length=max_length)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return answer

In [30]:
row_idx = 0
prompt = df.loc[row_idx, "prompt_low"]

raw = deepseek_generate_raw(prompt, model, tokenizer, device)
body = extract_clean_function_body(raw)

print("===== RAW GENERATION =====")
print(raw)

print("\n===== CLEAN BODY =====")
print(body)

print("\n===== FULL FUNCTION =====")
print(prompt + body)

Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.


===== RAW GENERATION =====
from typing import List

def has_close_elements(arg0: List[float], arg1: float) -> bool:
    """ Check if in given list of arg0, are any two arg0 closer to each other than
    given arg1.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """
    return False


if __name__ == "__main__":
    import doctest
    doctest.testmod()

===== CLEAN BODY =====
    return False


===== FULL FUNCTION =====
from typing import List

def has_close_elements(arg0: List[float], arg1: float) -> bool:
    """ Check if in given list of arg0, are any two arg0 closer to each other than
    given arg1.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """
    return False



In [31]:
df["deepseek_solution_original"] = ""

for i in range(len(df)):
    prompt = df.loc[i, "prompt_original"]
    raw = deepseek_generate_raw(prompt, model, tokenizer, device)
    body = extract_clean_function_body(raw)
    df.loc[i, "deepseek_solution_original"] = body

print("Generation complete.")

Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end gene

Generation complete.


In [32]:
df["deepseek_solution_low"] = ""

for i in range(len(df)):
    prompt = df.loc[i, "prompt_low"]
    raw = deepseek_generate_raw(prompt, model, tokenizer, device)
    body = extract_clean_function_body(raw)
    df.loc[i, "deepseek_solution_low"] = body

print("Generation complete.")

Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end gene

Generation complete.


In [33]:
df["deepseek_solution_high"] = ""

for i in range(len(df)):
    prompt = df.loc[i, "prompt_high"]
    raw = deepseek_generate_raw(prompt, model, tokenizer, device)
    body = extract_clean_function_body(raw)
    df.loc[i, "deepseek_solution_high"] = body

print("Generation complete.")

Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end gene

Generation complete.


In [34]:
df.head()

,task_id,prompt,canonical_solution,test,entry_point,prompt_original,prompt_low,prompt_high,codet5_solution_original,codet5_solution_low,codet5_solution_high,deepseek_solution_original,deepseek_solution_low,deepseek_solution_high
0,HumanEval/0,from typing import List\n\n\ndef has_close_ele...,"for idx, elem in enumerate(numbers):\n ...","\n\nMETADATA = {\n 'author': 'jt',\n 'da...",has_close_elements,from typing import List\n\n\ndef has_close_ele...,from typing import List\n\ndef has_close_eleme...,from typing import List\n\ndef has_close_eleme...,for i in range(len(numbers) - 1):\n ...,for i in range(len(arg0)):\n for j ...,for i in range(len(arg0)):\n ...,for i in range(len(numbers)):\n for...,return False\n,return False\n
1,HumanEval/1,from typing import List\n\n\ndef separate_pare...,result = []\n current_string = []\n ...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",separate_paren_groups,from typing import List\n\n\ndef separate_pare...,from typing import List\n\ndef separate_paren_...,from typing import List\n\ndef separate_paren_...,"paren_string = paren_string.replace(' ', '...",paren_groups = []\n paren_stack = []\n ...,paren_stack = []\n paren_list =...,paren_stack = []\n paren_list = []\n ...,# TODO: Write your code here\n pass\n,"assert separate_paren_groups(""(a)"") == [""a..."
2,HumanEval/2,\n\ndef truncate_number(number: float) -> floa...,return number % 1.0\n,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",truncate_number,\n\ndef truncate_number(number: float) -> floa...,def truncate_number(arg0: float) -> float:\n ...,def truncate_number(arg0: float) -> float:\n ...,if number == 0:\n return 0\n if ...,return arg0 - int(arg0)\n,return arg0\n,return number - int(number)\n,return arg0 - int(arg0)\n,return trunc(arg0)\n
3,HumanEval/3,from typing import List\n\n\ndef below_zero(op...,balance = 0\n\n for op in operations:\n...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",below_zero,from typing import List\n\n\ndef below_zero(op...,from typing import List\n\ndef below_zero(arg0...,from typing import List\n\ndef below_zero(arg0...,for operation in operations:\n if o...,if arg0[0] < 0:\n return True\n ...,if arg0[0] < 0:\n return Tr...,balance = 0\n for operation in operatio...,return False\n,if sum(arg0) < 0:\n return True\n ...
4,HumanEval/4,from typing import List\n\n\ndef mean_absolute...,mean = sum(numbers) / len(numbers)\n re...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",mean_absolute_deviation,from typing import List\n\n\ndef mean_absolute...,from typing import List\n\ndef mean_absolute_d...,from typing import List\n\ndef mean_absolute_d...,mean = sum(numbers) / len(numbers)\n re...,arg0_mean = sum(arg0) / len(arg0)\n ret...,return sum(abs(x - arg0[0]) for x in a...,mean = sum(numbers) / len(numbers)\n re...,return 0.0\n,return 0.0\n


In [35]:
out_path = "./data/openai_humaneval/humaneval_completed.jsonl"

df.to_json(out_path, orient="records", lines=True)
print("Saved to:", out_path)

Saved to: ./data/openai_humaneval/humaneval_completed.jsonl
